In [11]:
import pandas as pd
import numpy as np
import zipfile
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import datetime
import urllib.request, urllib.error
import os.path
import datetime

In [12]:
all_deaths_path = "data/sco_covid_data/custom_data/all_deaths_lad_18.csv"
covid_deaths_path = "data/sco_covid_data/custom_data/covid_deaths_lad_18.csv"
loc_deaths_all_path = "data/sco_covid_data/custom_data/deaths_by_location_all_lad_18.csv"
loc_deaths_corona_path = "data/sco_covid_data/custom_data/deaths_by_location_corona_lad_18.csv"
path = "https://www.nrscotland.gov.uk/files//statistics/covid19/"

In [13]:
def get_week_num():
    return datetime.date.today().isocalendar()[1]

In [23]:
'''NEED TO REFACTOR THIS'''
def get_covid_deaths_df(path):
            
    # Try get get the most recent data
    try:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-1)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        df_all_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 2 - All deaths.csv'),header = 3, encoding='unicode-escape')
        
        df_covid_deaths_trimmed = df_covid_deaths.iloc[48:80, 1:get_week_num()+1]
        df_all_deaths_trimmed = df_all_deaths.iloc[50:82, 1:get_week_num()+1]

        
        return df_all_deaths_trimmed, df_covid_deaths_trimmed
    
    # If error, then get last week response
    except urllib.error.HTTPError as e:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-2)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        
        idx_start = np.where(df_covid_deaths['Unnamed: 1'] == 'Aberdeen City')[0][0]
        idx_end = idx_start + 32
        df_covid_deaths_trimmed = df_covid_deaths.iloc[idx_start:idx_end,1:get_week_num()]
        df_covid_deaths_trimmed = df_covid_deaths_trimmed.set_index('Unnamed: 1')
        
        return df_covid_deaths_trimmed

In [25]:
def get_death_locations_df(path, week):
            
    # Try get get the most recent data
    try:
        url = path + "covid-deaths-data-week-%d.zip" % week
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_location = pd.read_csv(zipfile.open('covid-deaths-data-week-%d_Table 3 - deaths by location.csv' % week), header=3, encoding='unicode-escape')
        idx_start = np.where(df_location['Unnamed: 0'] == 'Aberdeen City')[0][0]
        idx_end = idx_start + 32
        df_location_trimmed = df_location.iloc[idx_start:idx_end, :-2]

        
        return df_location_trimmed
    
    # If error, then get last week response
    except urllib.error.HTTPError as e:
        url = path + "covid-deaths-data-week-%d.zip" % week-1
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        
        df_location = pd.read_csv(zipfile.open('covid-deaths-data-week-%d_Table 3 - deaths by location.csv' % week), header=3, encoding='unicode-escape')

        idx_start = np.where(df_location['Unnamed: 0'] == 'Aberdeen City')[0][0]
        print(idx_start)
        idx_end = idx_start + 32
        df_location_trimmed = df_location.iloc[idx_start:idx_end, :-2]
        return df_location_trimmed

In [27]:
df_death_locations_trimmed = get_death_locations_df(path, 17)
df_death_locations_trimmed.head()

,Unnamed: 0,Care\nHome,Home / Non-institution,Hospital,Other\ninstitution3,All locations,Unnamed: 6,Care\nHome.1,Home / Non-institution.1,Hospital.1,Other\ninstitution3.1,All locations.1
24,Aberdeen City,22.0,3.0,23,0.0,48,NaN,208,214,403,0.0,825
25,Aberdeenshire,38.0,9.0,28,0.0,75,NaN,289,267,445,1.0,"1,002"
26,Angus,26.0,3.0,13,0.0,42,NaN,141,167,224,0.0,532
27,Argyll and Bute,13.0,6.0,23,0.0,42,NaN,91,163,186,0.0,440
28,City of Edinburgh,116.0,13.0,88,0.0,217,NaN,605,452,748,0.0,"1,805"


In [42]:
df_death_locations_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 24 to 55
Data columns (total 12 columns):
Unnamed: 0                  32 non-null object
Care
Home                   32 non-null float64
Home / Non-institution      32 non-null float64
Hospital                    32 non-null object
Other
institution3          32 non-null float64
All locations               32 non-null object
Unnamed: 6                  0 non-null float64
Care
Home.1                 32 non-null object
Home / Non-institution.1    32 non-null object
Hospital.1                  32 non-null object
Other
institution3.1        32 non-null float64
All locations.1             32 non-null object
dtypes: float64(5), object(7)
memory usage: 3.1+ KB


In [95]:
def get_data_for_location(df_locations_trimmed, area_name):
    
    df_area = df_locations_trimmed.set_index('Unnamed: 0')
    df_area_solo = df_area.loc[[area_name]]
    df_area_solo2 = df_area_solo.iloc[:,0:3]
    df_transpose = df_area_solo2.transpose()
    
    return df_transpose

In [96]:
df_tester = get_data_for_location(df_death_locations_trimmed, 'Aberdeen City')
df_tester

Unnamed: 0,Aberdeen City
Care\nHome,22
Home / Non-institution,3
Hospital,23


In [77]:
df_tester.to_csv('tester.csv')